# __Exploratory Data Analysis__

In the following few cells, we load the necessary packages into Python, then load the csv files for both the test & train datasets into their own dataframes.

In [3]:
import numpy             as np
import pandas            as pd
import matplotlib        as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

from sklearn import pipeline
from sklearn import preprocessing
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import gen_features
import sklearn.preprocessing, sklearn.decomposition, \
       sklearn.linear_model,  sklearn.pipeline, \
       sklearn.metrics

np.__version__, pd.__version__, mpl.__version__, sns.__version__

Out[ 1 ]: ('1.14.3', '0.18.1', '2.1.1', '0.8.1')

In [4]:
ais_train_df = pd.read_csv('/dbfs/mnt/group-ma755/data/ais-train.csv', dtype={'vessel_id': str})
ais_test_df = pd.read_csv('/dbfs/mnt/group-ma755/data/ais-test.csv')


For the next few cells we will take a look at a few of the columns using the .describe command.

In [6]:
ais_train_df[['Status']].describe()

Out[ 3 ]: 
 Status
count 983171
unique 3
top Laden
freq 355289

The highest occuring status is "Laden".  That makes sense as they ships would spend much more time at sea than in port.  As for "Ballast" status, one reason that might occur less often is if the ship heads to a different, close port to resupply.

In [8]:
ais_train_df[['Average']].describe()

Out[ 4 ]: 
 Average
count 983171.000000
mean 11936.696555
std 7553.002033
min 3793.000000
25% 6362.000000
50% 9493.000000
75% 14982.000000
max 42709.000000

The mean 'Average' amount is $11,936.  This is significantly lower than the maximum of $42,709.  Looking at the interquartile ranges above it seems that the $42K is an outlier.

In [10]:
ais_train_df[['dwt']].describe()

Out[ 5 ]: 
 dwt
count 983171.000000
mean 182780.113124
std 13235.380032
min 150400.000000
25% 176000.000000
50% 179400.000000
75% 181300.000000
max 233600.000000

The minimum dead weight tonnage, at 150,400, is very close the to lowest value for Capesize ships (150,000).  The maximum value for Capesize ships exceeds 400,000.  The maximum value in our dataset is only 233,600, so we're dealing a variety of ships in the lower end of the spectrum as far as dead weight tonnage is concerned.

Here we take a look at the data in general, just to get a brief overview.

In [13]:
ais_train_df

Out[ 6 ]: 
 Latitude Longitude Status vessel_id Date dwt Average
0 51.96561 4.11305 Port 33 2015-12-24 171200 4965
1 29.93672 121.88367 Port 227 2015-12-24 170200 4965
2 53.51803 9.93555 Port 953 2015-12-24 171100 4965
3 38.80103 118.24558 Port 1726 2015-12-24 180200 4965
4 -20.38154 116.59218 Port 2459 2015-12-24 177700 4965
5 -23.00062 -43.97166 Port 2463 2015-12-24 178000 4965
6 46.39970 31.11848 Port 2495 2015-12-24 180000 4965
7 40.84523 29.28262 Port 3030 2015-12-24 172600 4965
8 -20.28726 -40.11804 Port 3203 2015-12-24 177000 4965
9 -20.08289 118.61247 Port 3789 2015-12-24 178000 4965
10 -20.32662 118.57660 Port 5953 2015-12-24 173800 4965
11 -21.80765 -40.77078 Port 7604 2015-12-24 172600 4965
12 28.04453 -115.18188 Port 7625 2015-12-24 168900 4965
13 -20.63783 116.71800 Port 7634 2015-12-24 171400 4965
14 0.52663 117.70657 Port 7652 2015-12-24 177000 4965
15 1.33304 103.54933 Port 7696 2015-12-24 172600 4965
16 35.11710 119.41963 Port 7726 2015-12-24 172500 4965
17 29.93966 122.25977 Port 7810 2015-12-24 184900 4965
18 35.10450 119.80201 Port 7853 2015-12-24 170800 4965
19 51.91585 2.88635 Port 7927 2015-12-24 206300 4965
20 -20.09738 118.58445 Port 7971 2015-12-24 180200 4965
21 50.11687 -5.05830 Port 8049 2015-12-24 171000 4965
22 -28.44546 -71.22860 Port 8171 2015-12-24 203100 4965
23 34.72540 134.62775 Port 8250 2015-12-24 206300 4965
24 11.14012 -74.28162 Port 8338 2015-12-24 180200 4965
25 -20.52863 117.30822 Port 8385 2015-12-24 177000 4965
26 -21.13733 149.64610 Port 8422 2015-12-24 177000 4965
27 21.42308 108.39185 Port 8424 2015-12-24 177700 4965
28 1.30322 104.06647 Port 8562 2015-12-24 177700 4965
29 -20.57601 117.20238 Port 8681 2015-12-24 229200 4965
... ... ... ... ... ... ... ...
983141 12.12237 112.10828 Ballast 44632 2012-01-03 171700 24795
983142 -5.29590 117.25425 Ballast 44638 2012-01-03 171900 24795
983143 -28.18165 -17.87988 Ballast 44662 2012-01-03 180100 24795
983144 5.51017 94.17145 Ballast 44674 2012-01-03 182100 24795
983145 -20.65967 63.93867 Ballast 45472 2012-01-03 208000 24795
983146 1.98723 119.42537 Ballast 45763 2012-01-03 177600 24795
983147 41.54442 -24.00330 Ballast 45842 2012-01-03 180200 24795
983148 -54.25220 -74.58030 Ballast 45846 2012-01-03 180200 24795
983149 -16.89983 64.31417 Ballast 45931 2012-01-03 179300 24795
983150 -27.29960 42.94125 Ballast 45969 2012-01-03 180300 24795
983151 -26.77083 -25.64364 Ballast 46000 2012-01-03 229200 24795
983152 13.24312 120.35118 Ballast 46064 2012-01-03 180200 24795
983153 -35.08033 28.69855 Ballast 46347 2012-01-03 180200 24795
983154 -30.83117 1.78552 Ballast 46387 2012-01-03 203500 24795
983155 21.70697 121.22290 Ballast 46407 2012-01-03 206300 24795
983156 -9.54750 115.75963 Ballast 46425 2012-01-03 206300 24795
983157 35.62311 -29.05803 Ballast 46726 2012-01-03 182500 24795
983158 -14.38835 112.57516 Ballast 46954 2012-01-03 173000 24795
983159 16.36137 113.51105 Ballast 46991 2012-01-03 176000 24795
983160 10.08144 121.02429 Ballast 46994 2012-01-03 180300 24795
983161 14.05848 119.70995 Ballast 47001 2012-01-03 230000 24795
983162 -27.69038 70.39997 Ballast 47031 2012-01-03 176000 24795
983163 25.68013 -50.05678 Ballast 47032 2012-01-03 180400 24795
983164 -7.75472 150.41894 Ballast 47326 2012-01-03 178100 24795
983165 -15.45964 117.35011 Ballast 47336 2012-01-03 177800 24795
983166 20.87255 -31.92948 Ballast 47418 2012-01-03 172500 24795
983167 -7.26867 125.35427 Ballast 47785 2012-01-03 178800 24795
983168 4.79237 99.05423 Ballast 47907 2012-01-03 171000 24795
983169 20.90279 121.59267 Ballast 48005 2012-01-03 179400 24795
983170 30.93990 122.83512 Ballast 49371 2012-01-03 208000 24795

[983171 rows x 7 columns]

In the next cell, we calculate the average earnings per ship, sorted by the vessel_id.

In [15]:
ais_train_df[['Average','vessel_id']].groupby('vessel_id',sort=True).mean()

Out[ 7 ]: 
 Average
vessel_id 
100539 11865.111475
100666 11795.294691
100753 12127.676136
100796 11967.259338
101146 11970.706368
101714 11537.884615
1018 12057.924468
101804 11971.099768
101805 12008.957918
102070 11779.987500
102423 11798.689906
102469 11564.396083
102662 11894.096875
102829 11887.315401
102916 11842.290323
102920 11910.998849
102925 11849.298190
103023 12027.687570
103125 12279.104238
105911 11848.445556
105916 11828.254902
106230 11818.277650
10625 12037.991416
106314 11860.723473
10637 11849.971668
10655 11809.843750
10662 11617.778846
106856 11817.968008
106900 11895.988601
10691 12006.049266
... ...
8992 12037.988372
9003 11736.854214
9083 11756.158215
9133 11919.883938
9144 11932.622742
9214 11866.030457
9235 11366.964630
9258 11903.335397
933 11714.125000
9347 11928.498436
94 12097.422028
9403 12024.105263
9460 11635.426979
9484 11563.613762
9506 11736.607179
953 11896.814973
9531 11993.348502
9549 11830.900798
9564 11794.492912
9625 11903.555089
9633 12182.249400
9657 11856.531828
96677 11783.653289
96681 11825.785276
96685 11870.674004
96704 11753.177874
96734 11498.378238
9722 11845.721532
97440 11514.134189
97789 11998.645485

[1243 rows x 1 columns]

Next we again calculate the average earnings per ship, but this time we calculate the average earnings according to the status as well.

In [17]:
pd.pivot_table(ais_train_df, 
               values ='Average', 
               index  ='vessel_id', 
               columns='Status', 
               aggfunc=np.mean)

Out[ 8 ]: 
Status Ballast Laden Port
vessel_id 
100539 12736.436620 10650.024194 12624.111702
100666 11810.566406 11759.500000 11816.059490
100753 13780.475309 11962.727612 10421.770833
100796 12161.564784 11979.242857 11793.547753
101146 11421.253906 12223.738351 12194.552716
101714 11297.692308 12230.714286 11289.554707
1018 12695.731861 11608.401575 11930.169421
101804 13848.900332 10441.066879 11627.829960
101805 11181.433198 11706.757862 12898.005917
102070 13187.532872 10795.469849 11725.252747
102423 9898.526316 12883.745358 12241.769231
102469 12500.197917 10922.963680 11543.298165
102662 11891.515679 11701.972973 12190.845865
102829 12316.400794 10986.025641 12338.026042
102916 11412.957096 11453.884498 12707.637584
102920 9585.709091 9550.935374 16355.376667
102925 12324.383686 11552.018576 11634.449123
103023 11229.918440 12968.258953 11558.104839
103125 12614.670103 12944.298893 11385.479100
105911 11549.553398 11749.220290 12363.040650
105916 11235.190476 12083.845570 11978.857143
106230 11838.888000 12198.024931 11264.809339
10625 12785.290984 11387.225895 12203.796923
106314 13274.807560 10590.165680 11919.769737
10637 10958.842105 12493.441667 11356.858657
10655 12223.870079 10566.680617 13632.198413
10662 11904.397770 11310.214286 11666.885794
106856 10805.781609 13088.458462 11429.642298
106900 11842.151703 11783.254072 12051.208955
10691 11682.180505 12250.890728 12048.101333
... ... ... ...
8992 11848.221402 12566.340000 11456.483636
9003 12007.898230 11153.927681 12424.095618
9083 10361.066202 11668.464789 12780.968675
9133 13518.004739 11455.479058 11490.311828
9144 12554.522523 12559.253571 10766.314024
9214 11976.206081 12833.400641 10978.944297
9235 11880.415385 11089.432292 11273.799308
9258 13660.778210 11563.660839 11071.138498
933 12412.979757 11327.740831 11655.310484
9347 11738.780864 12646.616379 11667.617866
94 13295.065217 11123.769444 12168.473310
9403 12406.617978 12338.841360 11338.469453
9460 13256.529032 9485.439628 12362.337121
9484 11453.822951 12040.638177 11016.881633
9506 11839.309375 11074.379791 12163.766304
953 12999.623256 11105.368534 12405.125000
9531 10907.569343 12341.911854 12606.859060
9549 12344.455696 11100.424242 12108.565598
9564 12279.767918 12699.654902 10783.647696
9625 11120.530534 12115.540541 12279.424581
9633 13260.328125 12570.823729 10801.975265
9657 13716.771218 11211.421233 11088.277372
96677 11761.454819 11867.711039 11711.591440
96681 12893.864865 11127.448622 11512.638211
96685 13549.554656 11522.331565 11012.012121
96704 13299.120635 10006.227273 12467.287554
96734 11153.262821 11132.296399 12227.764706
9722 11718.389610 11667.463700 12345.004329
97440 12336.079310 10247.811644 11991.960000
97789 11713.914373 12770.037464 11215.838565

[1243 rows x 3 columns]

Interestingly, in many cases there seems to be little variation in the averages across the three statuses.  In some cases "Laden" has the lowest average for a ship, and in others it has the highest.

Next we convert the "Date" column from an object into a datetime variable using the Pandas function ".to_datetime".

In [20]:
ais_train_df['Date']=pd.to_datetime(ais_train_df.Date)

In [21]:
ais_train_df.sort_values(by=['Date'])

Out[ 10 ]: 
 Latitude Longitude Status vessel_id Date dwt Average
983170 30.93990 122.83512 Ballast 49371 2012-01-03 208000 24795
321939 35.34873 119.53431 Port 56426 2012-01-03 179200 24795
321940 31.49340 121.38899 Port 56431 2012-01-03 176400 24795
321941 37.58344 121.38571 Port 56432 2012-01-03 179000 24795
321942 35.33569 119.88423 Port 56608 2012-01-03 180000 24795
321943 31.97880 120.84922 Port 59820 2012-01-03 180000 24795
321944 -20.63703 116.71439 Port 59844 2012-01-03 180100 24795
321945 33.94807 130.82549 Port 59845 2012-01-03 181500 24795
321946 -28.92674 32.14847 Port 59846 2012-01-03 182300 24795
321947 11.11043 -74.28074 Port 59854 2012-01-03 179400 24795
321948 -20.40139 -40.11856 Port 59856 2012-01-03 182700 24795
321949 -21.18218 149.39486 Port 59857 2012-01-03 181500 24795
321950 21.72104 133.43630 Port 59871 2012-01-03 176800 24795
321951 -20.15265 118.52203 Port 60085 2012-01-03 179400 24795
321952 38.82296 118.19961 Port 60397 2012-01-03 179300 24795
321953 35.31641 119.88385 Port 60434 2012-01-03 181600 24795
321954 37.77115 120.12968 Port 60734 2012-01-03 180100 24795
321955 11.12479 -74.26871 Port 62384 2012-01-03 180100 24795
321956 -20.32673 118.57657 Port 62386 2012-01-03 208000 24795
321957 29.71182 122.56304 Port 62935 2012-01-03 179100 24795
321958 32.34353 135.17670 Port 63766 2012-01-03 182800 24795
321959 -20.37477 116.61154 Port 63773 2012-01-03 179900 24795
321960 53.74447 0.46921 Port 70763 2012-01-03 181500 24795
321961 30.86736 131.96830 Port 70769 2012-01-03 207900 24795
321962 34.94151 127.82268 Port 70850 2012-01-03 180200 24795
321963 31.34364 135.53035 Port 70872 2012-01-03 180700 24795
321964 -20.12323 118.54130 Port 71560 2012-01-03 175900 24795
321965 51.96433 4.11698 Port 73273 2012-01-03 177000 24795
321938 34.88718 119.98083 Port 56241 2012-01-03 175800 24795
321937 38.78305 118.31155 Port 56175 2012-01-03 180500 24795
... ... ... ... ... ... ... ...
677300 37.50172 145.44363 Ballast 8226 2015-12-24 206300 4965
677299 -8.52441 81.23812 Ballast 8139 2015-12-24 203300 4965
677298 4.49683 106.21255 Ballast 6471 2015-12-24 171600 4965
677297 13.14648 137.29027 Ballast 6208 2015-12-24 170000 4965
677316 -19.53353 118.36050 Ballast 11169 2015-12-24 177000 4965
677317 -25.74872 49.90383 Ballast 11290 2015-12-24 177000 4965
677318 22.46665 122.15713 Ballast 11294 2015-12-24 154100 4965
677319 -16.58940 116.56506 Ballast 11305 2015-12-24 180200 4965
677339 -19.65385 118.39335 Ballast 15510 2015-12-24 180200 4965
677338 29.73773 123.72027 Ballast 15458 2015-12-24 174200 4965
677337 21.70898 -17.56526 Ballast 15369 2015-12-24 164200 4965
677336 32.31600 -165.59050 Ballast 14438 2015-12-24 176300 4965
677335 23.84553 119.93311 Ballast 14428 2015-12-24 177700 4965
677334 -12.99965 116.32372 Ballast 14426 2015-12-24 177800 4965
677333 19.53118 -55.16445 Ballast 14425 2015-12-24 177200 4965
677332 12.78209 47.98996 Ballast 14412 2015-12-24 177900 4965
677331 7.23987 120.12843 Ballast 14404 2015-12-24 175900 4965
677341 26.15344 -34.48339 Ballast 15546 2015-12-24 177800 4965
677330 -17.54498 153.24883 Ballast 14292 2015-12-24 174700 4965
677328 -0.82531 118.94075 Ballast 12402 2015-12-24 175800 4965
677327 16.10500 115.38167 Ballast 12281 2015-12-24 174700 4965
677326 -3.09625 86.32293 Ballast 11693 2015-12-24 206300 4965
677325 -18.88188 118.98715 Ballast 11642 2015-12-24 180200 4965
677324 13.73903 119.99865 Ballast 11598 2015-12-24 206200 4965
677323 29.83032 123.48837 Ballast 11580 2015-12-24 203200 4965
677322 22.66754 146.33961 Ballast 11436 2015-12-24 206200 4965
677321 -19.39145 150.58128 Ballast 11432 2015-12-24 185900 4965
677320 44.67590 -49.48543 Ballast 11401 2015-12-24 176900 4965
677329 -33.82155 14.27965 Ballast 13824 2015-12-24 180200 4965
0 51.96561 4.11305 Port 33 2015-12-24 171200 4965

[983171 rows x 7 columns]

In [22]:
ais_train_df.sort(['Date'], ascending=False)

/tmp/1525452641674-0/PythonShell.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
 from __future__ import absolute_import
 Out[ 11 ]: 
 Latitude Longitude Status vessel_id Date dwt Average
0 51.96561 4.11305 Port 33 2015-12-24 171200 4965
322202 5.24423 93.64570 Laden 51617 2015-12-24 177000 4965
322193 -22.33620 86.40549 Laden 51162 2015-12-24 181500 4965
322194 30.89115 122.87814 Laden 51170 2015-12-24 180100 4965
322195 51.02492 170.28265 Laden 51194 2015-12-24 179400 4965
322196 54.55055 -136.80420 Laden 51208 2015-12-24 176000 4965
322197 30.41908 123.89603 Laden 51431 2015-12-24 171200 4965
322198 37.59714 9.61985 Laden 51437 2015-12-24 177000 4965
322199 -3.48111 118.32769 Laden 51578 2015-12-24 178100 4965
322200 14.51933 118.94300 Laden 51580 2015-12-24 230000 4965
322201 -28.05033 78.06322 Laden 51602 2015-12-24 176000 4965
322203 30.36033 123.80336 Laden 51647 2015-12-24 180200 4965
322214 -11.38058 79.44007 Laden 52370 2015-12-24 179400 4965
322204 -10.60046 -125.32752 Laden 51827 2015-12-24 177000 4965
322205 28.95225 32.82024 Laden 51861 2015-12-24 169200 4965
322206 -23.36424 -42.94884 Laden 51884 2015-12-24 177900 4965
322207 -37.97685 9.21268 Laden 51933 2015-12-24 176000 4965
322208 -26.25220 -37.69622 Laden 52049 2015-12-24 178400 4965
322209 -21.20268 -37.24535 Laden 52226 2015-12-24 203300 4965
322210 42.07280 -29.72843 Laden 52260 2015-12-24 179200 4965
322211 8.05163 120.12342 Laden 52274 2015-12-24 178100 4965
322212 -8.60811 115.81206 Laden 52354 2015-12-24 179400 4965
322192 8.97943 120.36432 Laden 51009 2015-12-24 206300 4965
322191 29.13133 -39.67117 Laden 50979 2015-12-24 172900 4965
322190 15.68958 135.41731 Laden 50959 2015-12-24 171200 4965
322189 -36.62438 11.71380 Laden 50941 2015-12-24 180200 4965
322170 12.47162 89.48765 Laden 49353 2015-12-24 179400 4965
322171 3.75832 119.56502 Laden 49371 2015-12-24 208000 4965
322172 -24.57676 -40.36922 Laden 49391 2015-12-24 182600 4965
322173 31.40717 123.33687 Laden 49405 2015-12-24 176000 4965
... ... ... ... ... ... ... ...
677098 -21.43013 154.03708 Laden 11011 2012-01-03 177300 24795
677126 -35.40733 2.23285 Laden 16592 2012-01-03 177200 24795
677099 -34.88575 48.15411 Laden 11088 2012-01-03 208000 24795
677100 14.08905 119.92963 Laden 11126 2012-01-03 177000 24795
677101 35.90662 -6.09973 Laden 11169 2012-01-03 177000 24795
677102 -2.69012 107.20395 Laden 11170 2012-01-03 206300 24795
677103 -5.81190 -105.64988 Laden 11250 2012-01-03 180200 24795
677104 4.65635 119.63098 Laden 11262 2012-01-03 176900 24795
677105 -33.41932 41.53525 Laden 11284 2012-01-03 176900 24795
677106 -35.07308 25.66182 Laden 11290 2012-01-03 177000 24795
677107 -6.01168 125.51535 Laden 11294 2012-01-03 154100 24795
677108 -24.28991 90.31504 Laden 11436 2012-01-03 206200 24795
677109 33.71359 132.39603 Laden 11482 2012-01-03 177000 24795
677110 -9.00583 151.58410 Laden 11519 2012-01-03 180200 24795
677111 10.07043 121.18328 Laden 11598 2012-01-03 206200 24795
677112 1.29233 104.25513 Laden 11636 2012-01-03 203300 24795
677113 18.88760 118.51787 Laden 12281 2012-01-03 174700 24795
677114 -17.78950 116.70082 Laden 12297 2012-01-03 174400 24795
677115 14.60485 -17.85690 Laden 13299 2012-01-03 171100 24795
677116 -7.33327 63.49827 Laden 13824 2012-01-03 180200 24795
677117 12.97853 120.18705 Laden 13879 2012-01-03 180200 24795
677118 -34.61610 22.84608 Laden 14004 2012-01-03 174300 24795
677119 0.26998 119.13598 Laden 14333 2012-01-03 175500 24795
677120 14.39710 119.81010 Laden 14342 2012-01-03 173900 24795
677121 -33.53502 33.41402 Laden 14476 2012-01-03 176300 24795
677122 -17.27452 117.78223 Laden 15458 2012-01-03 174200 24795
677123 3.40908 119.55765 Laden 15714 2012-01-03 207800 24795
677124 -27.49587 12.39828 Laden 15792 2012-01-03 169300 24795
677125 -4.74313 117.65592 Laden 16399 2012-01-03 207900 24795
983170 30.93990 122.83512 Ballast 49371 2012-01-03 208000 24795

[983171 rows x 7 columns]

In [23]:
import datetime as dt

In [24]:
ais_train_df['Date']= ais_train_df['Date'].apply(lambda x:x.date())

In [25]:
ais_train_df

Out[ 14 ]: 
 Latitude Longitude Status vessel_id Date dwt Average
0 51.96561 4.11305 Port 33 2015-12-24 171200 4965
1 29.93672 121.88367 Port 227 2015-12-24 170200 4965
2 53.51803 9.93555 Port 953 2015-12-24 171100 4965
3 38.80103 118.24558 Port 1726 2015-12-24 180200 4965
4 -20.38154 116.59218 Port 2459 2015-12-24 177700 4965
5 -23.00062 -43.97166 Port 2463 2015-12-24 178000 4965
6 46.39970 31.11848 Port 2495 2015-12-24 180000 4965
7 40.84523 29.28262 Port 3030 2015-12-24 172600 4965
8 -20.28726 -40.11804 Port 3203 2015-12-24 177000 4965
9 -20.08289 118.61247 Port 3789 2015-12-24 178000 4965
10 -20.32662 118.57660 Port 5953 2015-12-24 173800 4965
11 -21.80765 -40.77078 Port 7604 2015-12-24 172600 4965
12 28.04453 -115.18188 Port 7625 2015-12-24 168900 4965
13 -20.63783 116.71800 Port 7634 2015-12-24 171400 4965
14 0.52663 117.70657 Port 7652 2015-12-24 177000 4965
15 1.33304 103.54933 Port 7696 2015-12-24 172600 4965
16 35.11710 119.41963 Port 7726 2015-12-24 172500 4965
17 29.93966 122.25977 Port 7810 2015-12-24 184900 4965
18 35.10450 119.80201 Port 7853 2015-12-24 170800 4965
19 51.91585 2.88635 Port 7927 2015-12-24 206300 4965
20 -20.09738 118.58445 Port 7971 2015-12-24 180200 4965
21 50.11687 -5.05830 Port 8049 2015-12-24 171000 4965
22 -28.44546 -71.22860 Port 8171 2015-12-24 203100 4965
23 34.72540 134.62775 Port 8250 2015-12-24 206300 4965
24 11.14012 -74.28162 Port 8338 2015-12-24 180200 4965
25 -20.52863 117.30822 Port 8385 2015-12-24 177000 4965
26 -21.13733 149.64610 Port 8422 2015-12-24 177000 4965
27 21.42308 108.39185 Port 8424 2015-12-24 177700 4965
28 1.30322 104.06647 Port 8562 2015-12-24 177700 4965
29 -20.57601 117.20238 Port 8681 2015-12-24 229200 4965
... ... ... ... ... ... ... ...
983141 12.12237 112.10828 Ballast 44632 2012-01-03 171700 24795
983142 -5.29590 117.25425 Ballast 44638 2012-01-03 171900 24795
983143 -28.18165 -17.87988 Ballast 44662 2012-01-03 180100 24795
983144 5.51017 94.17145 Ballast 44674 2012-01-03 182100 24795
983145 -20.65967 63.93867 Ballast 45472 2012-01-03 208000 24795
983146 1.98723 119.42537 Ballast 45763 2012-01-03 177600 24795
983147 41.54442 -24.00330 Ballast 45842 2012-01-03 180200 24795
983148 -54.25220 -74.58030 Ballast 45846 2012-01-03 180200 24795
983149 -16.89983 64.31417 Ballast 45931 2012-01-03 179300 24795
983150 -27.29960 42.94125 Ballast 45969 2012-01-03 180300 24795
983151 -26.77083 -25.64364 Ballast 46000 2012-01-03 229200 24795
983152 13.24312 120.35118 Ballast 46064 2012-01-03 180200 24795
983153 -35.08033 28.69855 Ballast 46347 2012-01-03 180200 24795
983154 -30.83117 1.78552 Ballast 46387 2012-01-03 203500 24795
983155 21.70697 121.22290 Ballast 46407 2012-01-03 206300 24795
983156 -9.54750 115.75963 Ballast 46425 2012-01-03 206300 24795
983157 35.62311 -29.05803 Ballast 46726 2012-01-03 182500 24795
983158 -14.38835 112.57516 Ballast 46954 2012-01-03 173000 24795
983159 16.36137 113.51105 Ballast 46991 2012-01-03 176000 24795
983160 10.08144 121.02429 Ballast 46994 2012-01-03 180300 24795
983161 14.05848 119.70995 Ballast 47001 2012-01-03 230000 24795
983162 -27.69038 70.39997 Ballast 47031 2012-01-03 176000 24795
983163 25.68013 -50.05678 Ballast 47032 2012-01-03 180400 24795
983164 -7.75472 150.41894 Ballast 47326 2012-01-03 178100 24795
983165 -15.45964 117.35011 Ballast 47336 2012-01-03 177800 24795
983166 20.87255 -31.92948 Ballast 47418 2012-01-03 172500 24795
983167 -7.26867 125.35427 Ballast 47785 2012-01-03 178800 24795
983168 4.79237 99.05423 Ballast 47907 2012-01-03 171000 24795
983169 20.90279 121.59267 Ballast 48005 2012-01-03 179400 24795
983170 30.93990 122.83512 Ballast 49371 2012-01-03 208000 24795

[983171 rows x 7 columns]

The following cell shows the average dead weigh tonnage of all the ships operating on a particular day.

In [27]:
ais_train_df.groupby('Date')['dwt'].mean()

Out[ 15 ]: 
Date
2012-01-03 181206
2012-01-04 181205
2012-01-05 181357
2012-01-06 181196
2012-01-09 181278
2012-01-10 181004
2012-01-11 181252
2012-01-12 181078
2012-01-13 181382
2012-01-16 181296
2012-01-17 181214
2012-01-18 181166
2012-01-19 181197
2012-01-20 181493
2012-01-23 181077
2012-01-24 181377
2012-01-25 181595
2012-01-26 181461
2012-01-27 181333
2012-01-30 181216
2012-01-31 181343
2012-02-01 181303
2012-02-02 181102
2012-02-03 180962
2012-02-06 181256
2012-02-07 181337
2012-02-08 180952
2012-02-09 181330
2012-02-10 181362
2012-02-13 181650
 ... 
2015-11-13 183794
2015-11-16 183604
2015-11-17 183608
2015-11-18 183723
2015-11-19 183645
2015-11-20 183652
2015-11-23 183671
2015-11-24 183534
2015-11-25 183806
2015-11-26 183813
2015-11-27 183663
2015-11-30 183634
2015-12-01 183640
2015-12-02 183727
2015-12-03 183708
2015-12-04 183710
2015-12-07 183786
2015-12-08 183733
2015-12-09 183742
2015-12-10 183688
2015-12-11 183702
2015-12-14 183632
2015-12-15 183652
2015-12-16 183558
2015-12-17 183722
2015-12-18 183734
2015-12-21 183752
2015-12-22 183583
2015-12-23 183739
2015-12-24 183821
Name: dwt, dtype: int64

Here we take a look at the number of ships in service on a particular day.

In [29]:
ais_train_df.groupby('Date')['vessel_id'].count()

Out[ 16 ]: 
Date
2012-01-03 699
2012-01-04 693
2012-01-05 674
2012-01-06 660
2012-01-09 688
2012-01-10 684
2012-01-11 671
2012-01-12 671
2012-01-13 693
2012-01-16 713
2012-01-17 704
2012-01-18 719
2012-01-19 731
2012-01-20 722
2012-01-23 722
2012-01-24 732
2012-01-25 746
2012-01-26 726
2012-01-27 727
2012-01-30 702
2012-01-31 702
2012-02-01 714
2012-02-02 725
2012-02-03 725
2012-02-06 714
2012-02-07 727
2012-02-08 735
2012-02-09 723
2012-02-10 710
2012-02-13 718
 ... 
2015-11-13 1109
2015-11-16 1120
2015-11-17 1118
2015-11-18 1123
2015-11-19 1108
2015-11-20 1104
2015-11-23 1106
2015-11-24 1118
2015-11-25 1121
2015-11-26 1111
2015-11-27 1131
2015-11-30 1113
2015-12-01 1120
2015-12-02 1134
2015-12-03 1116
2015-12-04 1116
2015-12-07 1130
2015-12-08 1124
2015-12-09 1115
2015-12-10 1120
2015-12-11 1117
2015-12-14 1137
2015-12-15 1103
2015-12-16 1107
2015-12-17 1122
2015-12-18 1115
2015-12-21 1093
2015-12-22 1106
2015-12-23 1111
2015-12-24 1118
Name: vessel_id, dtype: int64

The following cell creates a new dataframe based on the ais_train_df dataframe.

In [31]:
ais_train_df2=ais_train_df.sort_values(by=['Date'])
ais_train_df2['Date']=pd.to_datetime(ais_train_df2.Date)
dates=ais_train_df2['Date']
average=ais_train_df2['Average']
ais_t_df=pd.concat([dates, average], axis=1)
ais_t_df

Out[ 17 ]: 
 Date Average
983170 2012-01-03 24795
321939 2012-01-03 24795
321940 2012-01-03 24795
321941 2012-01-03 24795
321942 2012-01-03 24795
321943 2012-01-03 24795
321944 2012-01-03 24795
321945 2012-01-03 24795
321946 2012-01-03 24795
321947 2012-01-03 24795
321948 2012-01-03 24795
321949 2012-01-03 24795
321950 2012-01-03 24795
321951 2012-01-03 24795
321952 2012-01-03 24795
321953 2012-01-03 24795
321954 2012-01-03 24795
321955 2012-01-03 24795
321956 2012-01-03 24795
321957 2012-01-03 24795
321958 2012-01-03 24795
321959 2012-01-03 24795
321960 2012-01-03 24795
321961 2012-01-03 24795
321962 2012-01-03 24795
321963 2012-01-03 24795
321964 2012-01-03 24795
321965 2012-01-03 24795
321938 2012-01-03 24795
321937 2012-01-03 24795
... ... ...
677300 2015-12-24 4965
677299 2015-12-24 4965
677298 2015-12-24 4965
677297 2015-12-24 4965
677316 2015-12-24 4965
677317 2015-12-24 4965
677318 2015-12-24 4965
677319 2015-12-24 4965
677339 2015-12-24 4965
677338 2015-12-24 4965
677337 2015-12-24 4965
677336 2015-12-24 4965
677335 2015-12-24 4965
677334 2015-12-24 4965
677333 2015-12-24 4965
677332 2015-12-24 4965
677331 2015-12-24 4965
677341 2015-12-24 4965
677330 2015-12-24 4965
677328 2015-12-24 4965
677327 2015-12-24 4965
677326 2015-12-24 4965
677325 2015-12-24 4965
677324 2015-12-24 4965
677323 2015-12-24 4965
677322 2015-12-24 4965
677321 2015-12-24 4965
677320 2015-12-24 4965
677329 2015-12-24 4965
0 2015-12-24 4965

[983171 rows x 2 columns]

Next we drop all the duplicate values from the newly created dataframe.

In [33]:
unique_t_df=ais_t_df.drop_duplicates()
unique_t_df.set_index('Date',inplace=True)

In [34]:
unique_t_df[0:]

Out[ 19 ]: 
 Average
Date 
2012-01-03 24795
2012-01-04 22697
2012-01-05 18729
2012-01-06 16424
2012-01-09 15498
2012-01-10 14368
2012-01-11 13129
2012-01-12 10999
2012-01-13 10170
2012-01-16 9562
2012-01-17 8869
2012-01-18 8191
2012-01-19 7898
2012-01-20 7783
2012-01-23 7726
2012-01-24 7239
2012-01-25 7074
2012-01-26 6843
2012-01-27 6679
2012-01-30 6605
2012-01-31 6496
2012-02-01 6444
2012-02-02 6430
2012-02-03 6370
2012-02-06 6333
2012-02-07 6362
2012-02-08 6352
2012-02-09 6360
2012-02-10 6346
2012-02-13 6426
... ...
2015-11-13 6280
2015-11-16 6135
2015-11-17 5968
2015-11-18 5597
2015-11-19 5280
2015-11-20 5211
2015-11-23 5967
2015-11-24 6409
2015-11-25 7062
2015-11-26 7682
2015-11-27 8396
2015-11-30 8486
2015-12-01 8977
2015-12-02 8715
2015-12-03 8189
2015-12-04 7863
2015-12-07 7522
2015-12-08 7609
2015-12-09 7486
2015-12-10 7181
2015-12-11 6799
2015-12-14 6341
2015-12-15 5514
2015-12-16 5044
2015-12-17 5008
2015-12-18 5190
2015-12-21 5156
2015-12-22 4958
2015-12-23 4897
2015-12-24 4965

[999 rows x 1 columns]

The next cell is a line graph representing the Average amount over the time frame of our dataset.  We can see that the highest earnings are between the end of 2013 and the first half of 2014.

In [36]:
unique_t_df.plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.xlabel('Date', fontsize=20)
display()

Next we smooth out the jumps by using a rolling average of the "Average" amount, using a time period of 60 days.

In [38]:
Average = unique_t_df[['Average']]
Average.rolling(60).mean().plot(figsize=(20,10), linewidth=5, fontsize=20)
plt.xlabel('Date', fontsize=20)
display()

In [39]:
Average.diff().plot(figsize=(20,10), linewidth=5, fontsize=20);
plt.xlabel('Date', fontsize=20);
display();

Now we will import the autocorrelation_plot function from Pandas, then present the Autocorrelation plot of the "Average" data.

In [41]:
from pandas.tools.plotting import autocorrelation_plot

In [42]:
plt.figure()
autocorrelation_plot(Average)
display()

In [43]:
location_t_df = pd.DataFrame(data= np.c_[ais_train_df['Latitude'], ais_train_df['Longitude'],ais_train_df['Status']],
                     columns= ['Latitude']+['Longitude'] + ['Status'])
location_t_df.head()

Out[ 25 ]: 
 Latitude Longitude Status
0 51.9656 4.11305 Port
1 29.9367 121.884 Port
2 53.518 9.93555 Port
3 38.801 118.246 Port
4 -20.3815 116.592 Port

In [44]:
sns.lmplot(x='Longitude', y='Latitude', fit_reg=False, data=location_t_df, col='Status')
display()

In [45]:
location_date_t_df=pd.DataFrame(data= np.c_[ais_train_df['Date'],ais_train_df['Latitude'], ais_train_df['Longitude'],ais_train_df['Status']],
                     columns= ['Date']+['Latitude']+['Longitude'] + ['Status'])

In [46]:
location_date_t_df.describe()

Out[ 54 ]: 
 Date Latitude Longitude Status
count 983171 983171.0000 983171.00000 983171
unique 999 789822.0000 796221.00000 3
top 2015-07-21 31.4695 116.71443 Laden
freq 1154 158.0000 141.00000 355289

In [47]:
location_date_t_df['Date'] = pd.to_datetime(location_date_t_df['Date']) 

In [48]:
sns.lmplot(x='Longitude', y='Latitude', fit_reg=False, data=location_date_t_df[location_date_t_df.Date== '2015-12-23'], col='Status')
display()

In [49]:
sns.lmplot(x='Longitude', y='Latitude', fit_reg=False, data=location_date_t_df[location_date_t_df.Date== '2015-07-21'], col='Status')
display()

__what are we doing in the next cell?__

In [51]:
a=round(ais_train_df.loc[ais_train_df['Status']== 'Port'],2)
len(a.Latitude.unique())

Out[ 27 ]: 6277

__FROM HERE__